# Tracking preserving manually annotated edges

In [ ]:
%pip install -q --upgrade -r requirements.txt

## Initialize napari

In [2]:
import napari
import numpy as np
import pandas as pd
from IPython.display import display
from skimage.io import imread
from skimage.measure import regionprops_table

from laptrack import LapTrack

In [3]:
images = imread("interactive_example_data/example_images.tif")
labels = imread("interactive_example_data/example_label.tif")

In [4]:
viewer = napari.Viewer()
viewer.add_image(images, name="images")
viewer.add_labels(labels, name="labels")

<Labels layer 'labels' at 0x17da3f3a0>

## Calculate properties of the segmentation

In [5]:
def calc_frame_regionprops(labels):  # noqa: E302
    dfs = []
    for frame in range(labels.shape[0]):
        df = pd.DataFrame(
            regionprops_table(labels[frame], properties=["label", "centroid"])
        )
        df["frame"] = frame
        dfs.append(df)
    return pd.concat(dfs)


regionprops_df = calc_frame_regionprops(labels)
display(regionprops_df.head())

,label,centroid-0,centroid-1,frame
0,3,103.091603,195.816794,0
1,25,151.812048,15.674699,0
2,35,31.552063,43.740668,0
3,59,60.389432,76.156556,0
4,64,125.100457,21.646119,0


# Tracking by LapTrack

## Tracking

In [6]:
lt = LapTrack(
    track_cost_cutoff=100**2, 
    splitting_cost_cutoff=20**2
)
track_df, split_df, merge_df = lt.predict_dataframe(
    regionprops_df,
    coordinate_cols=["centroid-0","centroid-1"],
    only_coordinate_cols=False
)

In [8]:
track_df

label  centroid-0  centroid-1  frame_y  tree_id  track_id
frame index                                                           
0     0          3  103.091603  195.816794        0        0         0
      1         25  151.812048   15.674699        0        1         1
      2         35   31.552063   43.740668        0        2         2
      3         59   60.389432   76.156556        0        3         3
      4         64  125.100457   21.646119        0        4         4
...            ...         ...         ...      ...      ...       ...
5     45       441  197.130952  127.000000        5       22        22
      46       443   95.888179   48.552716        5       42        54
      47       456   25.483645   40.707944        5        2         2
      48       457   32.871560   68.052752        5       19        19
      49       459    4.693182  191.176136        5       11        58

[273 rows x 6 columns]

## Adding the tracked data to the viewer

In [9]:
track_label_image = np.zeros_like(labels)
for (frame, _), row in track_df.iterrows():
    track_label_image[frame][labels[frame] == row["label"]] = row["track_id"] + 1

In [10]:
viewer.layers["labels"].visible = False
viewer.add_labels(track_label_image)

<Labels layer 'track_label_image' at 0x15c5b9060>

# Manual correction

add points for the cells validated maually (emurated)

In [11]:
manual_corrected = np.load("interactive_example_data/manual_corrected.npy")
viewer.add_points(manual_corrected, name="manually_validated_tracks")

<Points layer 'manually_validated_tracks' at 0x11869f430>

In [12]:
manual_corrected = viewer.layers["manually_validated_tracks"].data.astype(np.int16)
# you can also redraw the labels
new_labels = viewer.layers["track_label_image"].data
# get label values at the placed points
validated_track_labels = new_labels[tuple(manual_corrected.T)]
validated_frames = manual_corrected[:, 0]

In [13]:
validated_points = np.array(list(zip(validated_frames, validated_track_labels)))
validated_points = validated_points[np.argsort(validated_points[:, 0])]
validated_ind_pairs = [
    (((frame1, label1), (frame2, label2)))
    for ((frame1, label1), (frame2, label2)) in zip(
        validated_points[:-1], validated_points[1:]
    )
]
validated_ind_pairs

[((0, 34), (1, 34)),
 ((1, 34), (2, 6)),
 ((2, 6), (3, 4)),
 ((3, 4), (4, 9)),
 ((4, 9), (5, 9))]

# Second tracking preserving manually corrected data

In [14]:
new_regionprops_df = (
    calc_frame_regionprops(new_labels).set_index(["frame", "label"]).reset_index()
)
new_regionprops_df.loc[[33, 63, 95]]

,frame,label,centroid-0,centroid-1
33,0,34,108.448276,108.586207
63,1,28,119.007673,89.751918
95,2,14,131.883281,126.189274


In [ ]:
get_frame_index_pair = lambda frame, label: (
    frame,
    list(coord_labels[frame]).index(label),
)
validated_edges = [
    (get_frame_index_pair(frame1, label1), get_frame_index_pair(frame2, label2))
    for ((frame1, label1), (frame2, label2)) in validated_ind_pairs
]
validated_edges

pairs of the coordinates of the validated points

In [ ]:
lt = LapTrack(track_cost_cutoff=100**2, splitting_cost_cutoff=20**2)
new_tree = lt.predict(new_coords, connected_edges=validated_edges)
new_tracked_df, _, _ = convert_tree_to_dataframe(new_tree)

In [ ]:
new_track_label_image = np.zeros_like(new_labels)
for (frame, ind), row in new_tracked_df.iterrows():
    label = new_labels[frame] == coord_labels[frame][ind]
    new_track_label_image[frame][label] = row["track_id"] + 2
viewer.layers["track_label_image"].visible = False
viewer.add_labels(new_track_label_image)